# 0. Setup

Import packages and specify any important functions here.

In [1]:
# import standard python packages
import pandas as pd
import subprocess
import os
import dill
import datetime

# add the utils and env directories to the path
import sys
sys.path.append('../../utils/')
sys.path.append('../../env/')

# import functions from utils directory files
from string_functions import *
from biofile_handling import *

# import paths to software installs from env
from install_locs import *

# 1. Collect Dockets for each starting dataset
Collects Dockets for each of the species datasets in the `species_dict`.

This `dict` expects key:value pairs in the form of `species`:`conditions` and is passed to the MultiSpeciesDocket class, which creates a folder in `output/` for the analysis.

The folder name takes the following format:

- `species_concat` + `_` + `global_conditions` + `analysis_type`

Where `species_concat` is the string concatenation of the alphabetize list of species prefixes.

In [2]:
################
# general info #
################

species_dict = {
    'Danio_rerio': 'adultbrain',
    'Mus_musculus': 'adultbrain',
    'Xenopus_laevis': 'adultbrain'
}

global_conditions = 'adultbrain'

analysis_type = 'OrthoFinder'

################

sample_MSD = MultiSpeciesDocket(
    species_dict = species_dict,
    global_conditions = global_conditions,
    analysis_type = analysis_type
    )

sample_MSD.make_directory()
sample_MSD.get_Dockets()

# 2. Get the peptide file paths from each Docket

In [4]:
peptide_files = {pre: sample_MSD.species_Dockets[pre].transdecoder_pep for pre in sample_MSD.species_Dockets}
display(peptide_files)

for pepfile in peptide_files.values():
    start = os.path.abspath(pepfile.path)
    end = sample_MSD.directory + pepfile.filename
    if not os.path.exists(end):
        print('copying', start, 'to', end)
        subprocess.run(['cp', start, end])
    else:
        print('file', start, 'has already been moved')

{'Drer': <biofile_handling.TransdecoderOutFile at 0x7fb12caa0100>,
 'Mmus': <biofile_handling.TransdecoderOutFile at 0x7fb12caa1bd0>,
 'Xlae': <biofile_handling.TransdecoderOutFile at 0x7fb12caa34f0>}

file /home/ec2-user/glial-origins/output/Drer_adultbrain/GCF_000002035.5_GRCz10_genomic_cDNA.fna.transdecoder.pep has already been moved
file /home/ec2-user/glial-origins/output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic_cDNA.fna.transdecoder.pep has already been moved
copying /home/ec2-user/glial-origins/output/Xlae_adultbrain/XENLA_9.2_genome_cDNA.fa.transdecoder.pep to ../../output/DrerMmusXlae_adultbrain_OrthoFinder/XENLA_9.2_genome_cDNA.fa.transdecoder.pep


# 3. Run OrthoFinder on all collected peptide files

Before running, will check if analysis has already been done - if so, continues.

In [5]:
OF_folder = sample_MSD.directory + 'OrthoFinder/'
today = datetime.date.today()
datefolder = OF_folder + 'Results_' + today.strftime('%b%d')
print(datefolder)

if not os.path.exists(OF_folder):
    subprocess.run([ORTHOFINDER_LOC, '-f', sample_MSD.directory])
    print('running test')
else:
    print('OF folder already exists at', OF_folder)
    print('Checking for results folder')
    folders = os.listdir(OF_folder)
    datefolder = OF_folder + [i for i in folders if 'Results' in i][0]
    print('Results folder found at', datefolder)

orthogroups_file = OrthoFinderOutputFile(
    filename = '/Orthogroups/Orthogroups.tsv', 
    multispeciesdocket = sample_MSD,
    directory = datefolder)

orthogroups_df = pd.read_csv(orthogroups_file.path, sep = '\t')
display(orthogroups_df)

../../output/DrerMmusXlae_adultbrain_OrthoFinder/OrthoFinder/Results_Sep28

OrthoFinder version 2.5.4 Copyright (C) 2014 David Emms

2022-09-28 18:25:16 : Starting OrthoFinder 2.5.4
16 thread(s) for highly parallel tasks (BLAST searches etc.)
2 thread(s) for OrthoFinder algorithm

Checking required programs are installed
----------------------------------------
Test can run "mcl -h" - ok
Test can run "fastme -i /home/ec2-user/glial-origins/output/DrerMmusXlae_adultbrain_OrthoFinder/OrthoFinder/Results_Sep28/WorkingDirectory/SimpleTest.phy -o /home/ec2-user/glial-origins/output/DrerMmusXlae_adultbrain_OrthoFinder/OrthoFinder/Results_Sep28/WorkingDirectory/SimpleTest.tre" - ok

Dividing up work for BLAST for parallel processing
--------------------------------------------------
2022-09-28 18:25:18 : Creating diamond database 1 of 3
2022-09-28 18:25:19 : Creating diamond database 2 of 3
2022-09-28 18:25:19 : Creating diamond database 3 of 3

Running diamond all-versus-all
----------------

,Orthogroup,GCF_000001635.23_GRCm38.p3_genomic_cDNA.fna.transdecoder,GCF_000002035.5_GRCz10_genomic_cDNA.fna.transdecoder,XENLA_9.2_genome_cDNA.fa.transdecoder
0,OG0000000,NaN,NaN,"gene10395_t.p1, gene12148_t.p1, gene1307_t.p1,..."
1,OG0000001,rna75711.p2,"gene10036.p1, gene10201.p2, gene10535.p1, gene...",NaN
2,OG0000002,NaN,NaN,"gene2928_t.p3, gene38029_t.p2, gene41218_t.p1,..."
3,OG0000003,NaN,NaN,"gene264_t.p1, rna1157.p2, rna12598.p1, rna1516..."
4,OG0000004,NaN,NaN,"gene50425_t.p1, rna27145.p2, rna34455.p1, rna4..."
...,...,...,...,...
32348,OG0032348,NaN,NaN,"rna97220.p1, rna97221.p1"
32349,OG0032349,NaN,NaN,"rna97260.p1, rna97260.p2"
32350,OG0032350,NaN,NaN,"rna97447.p1, rna97448.p1"
32351,OG0032351,NaN,NaN,"rna97780.p1, rna97781.p2"


# 4. Generate Orthofinder_exc file from gxc file of each starting dataset

In [9]:
for pre in sample_MSD.species_Dockets.keys():
    
    exc_filename = sample_MSD.species_Dockets[pre].gxc.filename.replace('.' + sample_MSD.species_Dockets[pre].gxc.filetype, '_asOrthogroup.' + sample_MSD.species_Dockets[pre].gxc.filetype)
    
    exc = ExcFile(
            filename = exc_filename,
            sampledict = sample_MSD.SampleDicts[pre],
            gxcfile = sample_MSD.species_Dockets[pre].gxc,
            embedding = 'Orthogroup'
            )
                                                                        
    if os.path.exists(exc.path):
        print('Orthofinder_exc file already exists at', exc.path)
        sample_MSD.species_Dockets[pre].add_keyfile(exc, 'Orthogroup_exc')
        
        continue
                                                                        
    gtf_idmm_df = pd.read_csv(sample_MSD.species_Dockets[pre].gtf_idmm.path, index_col = 0, sep = '\t')
    species_column = sample_MSD.species_Dockets[pre].cdna.filename + '.transdecoder'
    
    orthogroups_df_copy = orthogroups_df.copy(deep = True)
    
    orthogroups_df_copy['protein_id'] = orthogroups_df_copy[species_column].str.split(', ')
    orthogroups_df_copy = orthogroups_df_copy.explode('protein_id')
    orthogroups_df_copy['transcript_id'] = orthogroups_df_copy['protein_id'].str.split('.', expand = True)[0]
    
    og_keys = orthogroups_df_copy[['transcript_id', 'protein_id', 'Orthogroup']].drop_duplicates()
    og_keys.dropna(inplace = True)
    
    og_idmm_df = gtf_idmm_df.merge(og_keys, on = 'transcript_id')
    
    og_idmm_filename = pre + '_' + sample_MSD.SampleDicts[pre].conditions + '_og-idmm.tsv'
    
    og_idmm = IdmmFile(
        filename = og_idmm_filename,
        sampledict = sample_MSD.SampleDicts[pre],
        kind = 'og_idmm',
        sources = [orthogroups_file, sample_MSD.species_Dockets[pre].gtf_idmm]
        )
    
    og_idmm_df.to_csv(og_idmm.path, sep = '\t', index = None)
    
    sample_MSD.species_Dockets[pre].add_keyfile(og_idmm, 'og_idmm')
    sample_MSD.species_Dockets[pre].add_keyfile(orthogroups_file, sample_MSD.species_concat + '_OGfile')
    
    og_gene_keys = og_idmm_df[['gene_name', 'Orthogroup']].drop_duplicates()
    
    gxc_df = pd.read_csv(sample_MSD.species_Dockets[pre].gxc.path, sep = '\t')
    gxc_original_dataname = gxc_df.columns[0]
    gxc_df.rename(columns = {gxc_original_dataname: 'gene_name'}, inplace = True)
    exc_df = og_gene_keys.merge(gxc_df, on = 'gene_name')
    
    exc_df = exc_df.drop(columns = 'gene_name')
    exc_df = exc_df.groupby('Orthogroup').agg({i: ('first' if i == 'Orthogroup' else 'sum') for i in exc_df.columns}).reset_index(drop = True)
    
    exc_df.to_csv(exc.path, sep = '\t', index = None)
    sample_MSD.species_Dockets[pre].add_keyfile(exc, 'Orthogroup_exc')
    
    display(exc_df)

,Orthogroup,ACAATATATTGTACCTGA,ACGTTGATGGCGTAGAGA,AACCTAACCTGAATTTGC,CTCGCAGCCCTCTATGTA,ACGTTGCGTATTTAGTCG,AACCTATAGAGACCGACG,ACGAGCGCTGTGGCCTAG,GCGAATGGACATGGACAT,TCTACCGCTCAAGCTCAA,...,CGGCAGTCAAAGATCTCT,GACACTGCGAATCTGTGT,GCAGGAGGCTGCTAAGGG,TATGTATACTTCCGCACC,TGGATGTTCCGCACAATA,AACCTATGGATGGGGTTT,AAGCGGAGGACTCTCCAT,ACCTGACTCGCAAGCGAG,ACGTTGCAAAGTTTCATA,ATCAACTGCAATTTCCGC
0,OG0000001,0,0,0,0,1,0,1,1,0,...,0,0,0,0,0,1,0,1,0,0
1,OG0000005,0,1,0,0,0,0,1,0,0,...,0,2,0,0,0,0,0,0,0,0
2,OG0000009,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,OG0000013,0,1,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4,OG0000014,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13612,OG0030138,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13613,OG0030139,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13614,OG0030140,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13615,OG0030141,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,Orthogroup,Brain_1.CCGCTAAATAAATAAGGG,Brain_1.AACGCCGATCTTGCCCTC,Brain_1.ACCTGAAGTTTATCGTAA,Brain_1.CTCGCACTGAAACCGCTA,Brain_1.ATCAACATCTCTTCGGGT,Brain_1.GCGAATAGGGTCTATGTA,Brain_1.CGAGTAAGGGTCTAGTCG,Brain_1.ATCTCTTCGTAAGTTGCC,Brain_1.AACGCCTAAGGGCTCGCA,...,Brain_1.TTGGACGCCTAGGAGATC,Brain_1.TTAACTAAAGTTTATGTA,Brain_1.GTCCCGGGACATAGGACT,Brain_1.CCGCTAGGGTTTGCTCAA,Brain_1.TGATCAGCTGTGTCAAAG,Brain_1.TCACTTGAATTATGAAGC,Brain_1.AAGTACGCTGTGTATGTA,Brain_1.CCTAGATAGAGAATTTGC,Brain_1.CATCCCATTTGCGGCTGC,Brain_1.CAAAGTGGGTTTAGCGAG
0,OG0000001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,OG0000005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,OG0000008,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,OG0000011,3,0,0,0,0,15,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,OG0000017,0,1,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18413,OG0028373,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18414,OG0028374,2,4,4,5,6,2,7,3,0,...,0,1,2,0,0,0,0,1,0,1
18415,OG0028375,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18416,OG0028377,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,Orthogroup,AACCTATTCATATAAGGG,CTCGCATCAAAGTTAACT,AACCTAGTATACTTCCGC,AACCTAAAAGTTCTGAAA,CTCGCACGCACCCTCCAT,ACGTTGTATTGTAGCGAG,ACGAGCATGCTTTAGTCG,AACCTAGTCCCGCCATCT,AACCTAGCGAATTAGAGA,...,TCACTTGTTGCCATGCTT,TCGGGTTGTCACACTTAT,TCGTAATCGTAAGTTGCC,TGTCACGAATTACACAAG,TGTGCGTACTTCTAGTCG,TTAACTATACAGTGGATG,TTGGACACTTATGATCTT,AAAGTTACTTATGCCCTC,AACCTACGCACCTGCGGA,AACCTATAGTCGCTGTGT
0,OG0000000,0,0,0,0,0,4,1,3,0,...,0,0,0,0,0,0,0,0,0,0
1,OG0000002,7,3,4,7,3,4,6,3,0,...,0,0,0,0,0,0,0,0,0,0
2,OG0000003,2,0,3,0,2,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,OG0000004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,OG0000005,2,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12244,OG0032312,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12245,OG0032313,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12246,OG0032314,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12247,OG0032318,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# 5. Generate merged exc file for all samples in dataset

(TO BE WRITTEN)

# 6. Pickle the species_Docket files

In [10]:
for pre in sample_MSD.species_Dockets.keys():

    dill_filename = sample_MSD.species_Dockets[pre].directory + '_'.join([pre, sample_MSD.species_Dockets[pre].conditions, 'sample_Docket.pkl'])

    with open(dill_filename, 'wb') as file:
        dill.dump(sample_MSD.species_Dockets[pre], file)